In [1]:
import cirq
import numpy as np

from cirq_sic import *

This modular addition algorithm works by temporarily calculating values like a+k. Since a and k can both be as large as d-1, their sum can reach up to (d-1) + (d-1) = 2d-2.

Your qubit register can only represent integers from 0 to $2^n-1$. For the algorithm's logic (especially the part that checks for overflow using the most significant bit) to be reliable, the temporary sum must not exceed the register's capacity.

This leads to the requirement: $2d-2 < 2^n$. Thus we can encode a qudit in $n$ qubits with $d$ satisfying

$$d \leq 2^{n-1}  $$

https://github.com/pifparfait/Efficient-Quantum-Modular-Arithmetics/blob/main/quantum_modular_artihmetics.ipynb

In [5]:
def sum_k(k, wires):
    for j, wire in enumerate(wires):
        angle = k * np.pi / (2**j)
        if angle != 0:
            yield cirq.ZPowGate(exponent=angle/np.pi)(wires[j])

In [6]:
def add_in_k_N(k, N, q, aux, is_in_standard_basis=True):
    wires_a = [aux[0]] + q
    if is_in_standard_basis:
        yield cirq.qft(*wires_a)
    yield from sum_k(k, wires_a)
    yield from cirq.inverse(list(sum_k(N, wires_a)))
    yield cirq.inverse(cirq.qft(*wires_a))
    yield cirq.CNOT(wires_a[0], aux[1])
    yield cirq.qft(*wires_a)
    yield from [op.controlled_by(aux[1]) for op in sum_k(N, wires_a)]
    yield from cirq.inverse(list(sum_k(k, wires_a)))
    yield cirq.inverse(cirq.qft(*wires_a))
    yield cirq.X(wires_a[0])
    yield cirq.CNOT(wires_a[0], aux[1])
    yield cirq.X(wires_a[0])
    yield cirq.qft(*wires_a)
    yield from sum_k(k, wires_a)
    if is_in_standard_basis:
        yield cirq.inverse(cirq.qft(*wires_a))

In [7]:
def add_in_N(N, wires_a, wires_b, aux, is_in_standard_basis=True):
    new_wires_b = [aux[0]] + wires_b
    if is_in_standard_basis:
        yield cirq.qft(*new_wires_b)
    for i, control_qubit in enumerate(wires_a):
        value = 2**(len(wires_a) - 1 - i)
        adder_ops = list(add_in_k_N(value, N, wires_b, aux, is_in_standard_basis=False))
        controlled_adder_ops = [op.controlled_by(control_qubit) for op in adder_ops]
        yield from controlled_adder_ops
    if is_in_standard_basis:
        yield cirq.inverse(cirq.qft(*new_wires_b))

def add_in_N_dag(N, wires_a, wires_b, aux, is_in_standard_basis=True):
    yield cirq.inverse(list(add_in_N(N, wires_a, wires_b, aux, is_in_standard_basis=is_in_standard_basis)))

In [87]:
def qft_in_N(d, qubits, inverse=False):
    if inverse:
        yield from cirq.inverse(qft_in_N(d, qubits))
        return
    n = len(qubits)
    F = np.array([[np.exp(2*np.pi*1j*i*j/d) for j in range(d)] for i in range(d)])/np.sqrt(d)
    Fd_gate = cirq.MatrixGate(sc.linalg.block_diag(F, np.eye(2**n - d)), name=f"DFT({d})")
    yield from cirq.decompose(cirq.Circuit((Fd_gate.on(*qubits))))

In [88]:
n = 2
d = 2**n
phi = rand_ket(d)
ansatz_preparation = ansatz_circuit(phi)

system_qubits = cirq.LineQubit.range(n)
ancilla_qubits = cirq.LineQubit.range(n, 2*n)
aux = [cirq.NamedQubit("aux1"), cirq.NamedQubit("aux2")]
sim = cirq.Simulator()

In [89]:
circ1 = cirq.Circuit((ansatz_preparation(system_qubits, conjugate=False),
                     cirq.qft(*system_qubits),
                     sum_k(1, system_qubits),
                     cirq.inverse(cirq.qft(*system_qubits))))
circ2 = cirq.Circuit((ansatz_preparation(system_qubits, conjugate=False),
                      X(system_qubits)))
circ3 = cirq.Circuit((ansatz_preparation(system_qubits, conjugate=False),
                      add_in_k_N(1, d, system_qubits, aux)))

res1, res2, res3 = sim.simulate(circ1),sim.simulate(circ2), sim.simulate(circ3)
rho1, rho2, rho3 = res1.density_matrix_of(system_qubits),\
                   res2.density_matrix_of(system_qubits),\
                   res3.density_matrix_of(system_qubits)
assert np.allclose(rho1, rho2)
assert np.allclose(rho2, rho3)

In [90]:
circ1 = cirq.Circuit((ansatz_preparation(ancilla_qubits, conjugate=True),
                      ansatz_preparation(system_qubits, conjugate=False),
                      CX(ancilla_qubits, system_qubits)))
circ2 = cirq.Circuit((ansatz_preparation(ancilla_qubits, conjugate=True),
                      ansatz_preparation(system_qubits, conjugate=False),
                      add_in_N(d, ancilla_qubits, system_qubits, aux)))
res1, res2 = sim.simulate(circ1), sim.simulate(circ2)
rho1, rho2 = res1.density_matrix_of(ancilla_qubits+system_qubits),\
             res2.density_matrix_of(ancilla_qubits+system_qubits)
assert np.allclose(rho1, rho2)

In [93]:
n = 3
d = 2**n
d_s = 3
phi = load_sic_fiducial(d_s)
ket = rand_ket(d_s)
embedded_phi = np.concatenate([phi, np.zeros(d-d_s)])
embedded_ket = np.concatenate([ket, np.zeros(d-d_s)])
prepare_fiducial = ansatz_circuit(embedded_phi)
prepare_system = ansatz_circuit(embedded_ket)

system_qubits = cirq.LineQubit.range(n)
ancilla_qubits = cirq.LineQubit.range(n, 2*n)
aux = [cirq.NamedQubit("aux1"), cirq.NamedQubit("aux2")]
sim = cirq.Simulator()

circ = cirq.Circuit((prepare_system(system_qubits, conjugate=False),
                     prepare_fiducial(ancilla_qubits, conjugate=True),
                     add_in_N_dag(d_s, ancilla_qubits, system_qubits, aux),
                     qft_in_N(d_s, ancilla_qubits, inverse=True)))
res = sim.simulate(circ)
V = dirac(res.state_vector());

00000000: -0.07+-0.15j: 0.027
00000100: -0.35+0.26j: 0.190
00001000: -0.15+0.17j: 0.051
00100000: -0.11+-0.34j: 0.129
00100100: -0.33+-0.29j: 0.190
00101000: -0.19+0.09j: 0.046
01000000: -0.13+-0.48j: 0.244
01000100: 0.03+-0.10j: 0.011
01001000: -0.00+-0.33j: 0.112


In [94]:
E = wh_povm(phi)
change_conjugate_convention(np.array([ket.conj() @ e @ ket for e in E]).real)

array([0.027, 0.19 , 0.051, 0.129, 0.19 , 0.046, 0.244, 0.011, 0.112])